In [4]:
import numpy as np
import pandas as pd
import os
import shutil
from scipy import stats
df = pd.read_excel('IAPS_scores.xls')
df['图片编号'] = df.图片编号.astype(str)

df

,图片内容,图片编号,效价,效价标准差,唤醒度,唤醒度标准差,优势度,优势度标准差
0,Snake,1019,2.812500,1.864064,5.895833,2.553592,6.020833,2.383361
1,Snake,1022,4.403846,1.683363,5.423077,1.861332,5.846154,1.649535
2,Snake,1030,2.977273,1.302746,5.809524,2.166689,6.380952,1.637255
3,Snake,1040,2.454545,1.606014,5.642857,2.620942,6.690476,1.981476
4,Snake,1050,2.408163,1.619135,6.729167,2.180568,6.479167,2.221650
...,...,...,...,...,...,...,...,...
699,Auto accident,4664_2,3.541667,2.405299,6.265306,1.987204,5.479167,2.315439
700,Car accident,6250_1,2.903846,1.537245,5.730769,1.869417,5.826923,2.229228
701,Firefighter,6250_2,6.065217,2.254893,5.826087,1.912582,3.978261,2.275367
702,Auto accident,6570_1,2.521739,1.974719,6.152174,2.449194,6.565217,2.381896


In [5]:
files = os.listdir('IMAGES')
new_list = []
for item in files:
    new_list.append(item.split('.')[0])

new_list = pd.Series(new_list)
df_cut =df[df['图片编号'].isin(new_list) ]
df_cut = df_cut.sort_values(['效价'],ascending=False).reset_index()
# 列表专用：isin()
df_cut

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'IMAGES'

图片
![](source/negative/2385.JPG)

**requirements:**
>(i) positive, neutral and negative stimuli significantly differed among each other in valence ratings   (positive > neutral > negative)

>(ii) positive and negative stimuli were equally arousing while neutral ones were significantly less arousing (neutral < positive = negative)

> They performed two blocks for each emotional valence (negative, neutral, positive) for each cognitive load condition (1-back, 2-back) (total = 12 blocks).

In [2]:
df_positive = df_cut[:150].reset_index()
df_negative = df_cut[-150:].reset_index()
df_neutral = df_cut[int((len(df_cut)-1)/2)-75:int((len(df_cut)-1)/2)+75].reset_index()

df_positive.drop(['index'],axis=1,inplace=True)
df_neutral.drop(['index'],axis=1,inplace=True)
df_negative.drop(['index'],axis=1,inplace=True)

df_positive['label'] = 'positive'
df_neutral['label'] = 'neutral'
df_negative['label'] = 'negative'

df_neutral

NameError: name 'df_cut' is not defined

In [52]:
def copy_files(folder,df):
    for poimter in range(len(df)):
        picname = df.图片编号[poimter]+'.JPG'

        shutil.copy(('IMAGES/'+ picname), (folder+picname))
    pass

# copy_files('source/negative/', df_negative)
# copy_files('source/positive/', df_positive)
# copy_files('source/neutral/', df_neutral)



In [53]:
def T_test(nplist_1, nplist_2):
    return stats.ttest_rel(nplist_1, nplist_2)

T_test(df_positive['效价'], df_negative['效价'])  # 差异显著

Ttest_relResult(statistic=200.51938332341783, pvalue=3.7740974287812285e-183)

In [54]:
T_test(df_positive['效价'], df_neutral['效价'])

Ttest_relResult(statistic=110.56526695266581, pvalue=6.736415400218894e-145)

In [55]:
print(len(df_positive), len(df_negative), len(df_neutral))

150 150 150


In [56]:
listr = os.listdir()
ner = []
for itrm in listr:
    if itrm.endswith('.xlsx'):
        ner.append(itrm)
table = ner[1:]
# d_exp = pd.read_excel(table[1])
table

['nag_1.xlsx',
 'nag_2.xlsx',
 'nag_3.xlsx',
 'neu_1.xlsx',
 'neu_2.xlsx',
 'neu_3.xlsx',
 'pos_1.xlsx',
 'pos_2.xlsx',
 'pos_3.xlsx',
 'practice_nag_1.xlsx',
 'practice_nag_2.xlsx',
 'practice_nag_3.xlsx',
 'practice_neu_1.xlsx',
 'practice_neu_2.xlsx',
 'practice_neu_3.xlsx',
 'practice_pos_1.xlsx',
 'practice_pos_2.xlsx',
 'practice_pos_3.xlsx']

In [57]:
start_location = 'source/'
for nume in table:
    length = 0

    df = pd.read_excel(nume)

    if 'practice' in nume:
        length = 10
    else:
        if '1' in nume:
            # 1-back
            length = 21
        elif '2' in nume:
            # 2-back
            length = 22
        else:
            # 3-back
            length = 23

    # change the picture location
    if 'nag' in nume:
        print(length)
        start_location = 'source/negative/'
        pn = df_negative.sample(length)
        df_negative = pd.concat([df_negative,pn,pn]).drop_duplicates(keep=False)
        pn = pn.reset_index()
        cn = start_location + pn.图片编号+".JPG"

        
    elif 'neu' in nume: 
        print(length)
        start_location = 'source/neutral/' 
        pt = df_neutral.sample(length)
        df_neutral = pd.concat([df_neutral,pt,pt]).drop_duplicates(keep=False)
        pt = pt.reset_index()
        cn = start_location + pt.图片编号+".JPG"

    else:
        print(length)
        start_location = 'source/positive/'
        pp = df_positive.sample(length)
        df_positive = pd.concat([df_positive,pp,pp]).drop_duplicates(keep=False)
        pp = pp.reset_index()
        cn = start_location + pp.图片编号+".JPG"
    
    df.Pic = cn
    df.to_excel(nume, index=False)

print(len(df_negative), len(df_neutral), len(df_positive))

21
22
23
21
22
23
21
22
23
10
10
10
10
10
10
10
10
10
54 54 54


In [2]:
import os
len(os.listdir('source/negative'))

90